In [3]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [6]:
# Setup plot defaultrialsrialsrialss
import matplotlib.pyplot as plt
import seaborn as sns

# Reset styling to default
plt.rcdefaults()
# Grid on
plt.rcParams['axes.grid'] = True
# Top and right axis spines off
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

# Title 18 x y labels 16 ticks 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
fig_w=6
fig_h=6
plt.rcParams['figure.figsize'] = (fig_w,fig_h )
# Make axis equal
#plt.rcParams['axes.axisbelow'] = True 


plt.rcParams['grid.color'] = '0.8'# Grid color

# Set default color cycle
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#043908', '#ff7f0e', '#9467bd', '#d62728', '#364704', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
# Set default grid settings
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.alpha'] = 0.7
plt.rcParams['font.family'] = 'sans-serif'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 70

In [ ]:
#load data
dataName="_bimodal_audioVisual_durEst_2025-03-18_14h49.07.357.csv"

data = pd.read_csv("data/"+dataName)

In [7]:
data

,Unnamed: 0,standardDur,riseDur,order,preDur,postDur,isiDur,trial_num,totalDur,delta_dur_percents,...,stair_is_reversal,response_keys,conflictDur,recordedOnsetVisualTest,recordedOffsetVisualTest,recordedDurVisualTest,recordedOnsetAudio,recordedOffsetAudio,recordedDurAudio,modalityPostCue
0,0,0.5,0.0,1,0.4250,0.4833,0.5833,0,2.8916,0.80,...,False,left,0.0,0.548692,1.463227,0.914535,2.797414,3.682409,0.884996,NaN
1,1,0.5,0.0,2,0.6083,0.5083,0.6750,1,3.1166,0.65,...,False,left,0.0,1.960931,2.793267,0.832336,0.598897,1.099184,0.500286,NaN
2,2,0.5,0.0,1,0.5917,0.5167,0.5333,2,3.0667,0.85,...,False,left,0.0,0.614293,1.547615,0.933321,3.230671,4.166023,0.935353,NaN
3,3,0.5,0.0,2,0.5417,0.5750,0.8333,3,2.6250,-0.65,...,False,right,0.0,3.145320,3.328779,0.183460,0.846002,1.528953,0.682951,NaN


In [8]:
from scipy.stats import norm
from scipy.optimize import minimize